<a href="https://colab.research.google.com/github/Shukla070/Utkarsh_231AI040_ML/blob/main/231AI040.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install sec-api


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip -q install pydash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
from sec_api import InsiderTradingApi
import pandas as pd
import math
from pydash import get, flatten

insiderTradingApi = InsiderTradingApi("OPENAI_KEY")

In [ ]:

query_string = "issuer.tradingSymbol:TSLA"

insider_trades_sample = insiderTradingApi.get_data({
  "query": {"query_string": {"query": query_string}},
  "from": "0",
  "size": "2",
  "sort": [{ "filedAt": { "order": "desc" } }]
})

In [ ]:
insider_trades_sample

{'total': {'value': 765, 'relation': 'eq'},
 'transactions': [{'id': '4be52e62da6b5e3f027f90e26b285158',
   'accessionNo': '0000950170-24-087441',
   'filedAt': '2024-07-29T19:00:04-04:00',
   'schemaVersion': 'X0508',
   'documentType': '4',
   'periodOfReport': '2024-07-25',
   'notSubjectToSection16': False,
   'issuer': {'cik': '1318605',
    'name': 'Tesla, Inc.',
    'tradingSymbol': 'TSLA'},
   'reportingOwner': {'cik': '1242782',
    'name': 'DENHOLM ROBYN M',
    'address': {'street1': 'C/O TESLA, INC.',
     'street2': '1 TESLA ROAD',
     'city': 'AUSTIN',
     'state': 'TX',
     'zipCode': '78725'},
    'relationship': {'isDirector': True,
     'isOfficer': False,
     'isTenPercentOwner': False,
     'isOther': False}},
   'nonDerivativeTable': {'transactions': [{'securityTitle': 'Common Stock',
      'transactionDate': '2024-07-25',
      'coding': {'formType': '4', 'code': 'M', 'equitySwapInvolved': False},
      'amounts': {'shares': 44096,
       'pricePerShare': 23.1

In [ ]:
def flatten_filing(filing):
  transactions = []


  try:
    base_data = {
        "periodOfReport": filing["periodOfReport"],
        "issuerCik": filing["issuer"]["cik"],
        "issuerTicker": filing["issuer"]["tradingSymbol"],
        "reportingPersonName": get(filing, "reportingOwner.name", ""),
        "reportingPersonCik": get(filing, "reportingOwner.cik", ""),
        "relationship": get(filing, "reportingOwner.relationship", {})
    }
  except Exception as e:
    print(f'{filing["id"]}, caught {type(e)}: {e}')
    return transactions

  if "derivativeTable" in filing and "transactions" in filing["derivativeTable"]:

    for transaction in filing["derivativeTable"]["transactions"]:
      shares = get(transaction, "amounts.shares", 0)
      sharePrice = get(transaction, "amounts.pricePerShare", 0)
      sharesOwnedFollowingTransaction = get(transaction, "postTransactionAmounts.sharesOwnedFollowingTransaction", 0)
      codingCode = get(transaction, "coding.code", "")
      underlyingSecurity = get(transaction, "underlyingSecurity.title", "")

      entry = {
          "type": "derivative",
          "securityTitle": transaction["securityTitle"],
          "underlyingSecurity": underlyingSecurity,
          "codingCode": codingCode,
          "acquiredDisposed": transaction["amounts"]["acquiredDisposedCode"],
          "shares": shares,
          "sharePrice": sharePrice,
          "total": math.ceil(shares * sharePrice),
          "sharesOwnedFollowingTransaction": sharesOwnedFollowingTransaction
      }


      transactions.append({**base_data, **entry})

  if "nonDerivativeTable" in filing and "transactions" in filing["nonDerivativeTable"]:

    for transaction in filing["nonDerivativeTable"]["transactions"]:
      sharePrice = get(transaction, "amounts.pricePerShare", 0)
      sharesOwnedFollowingTransaction = get(transaction, "postTransactionAmounts.sharesOwnedFollowingTransaction", 0)

      entry = {
          "type": "nonDerivative",
          "securityTitle": transaction["securityTitle"],
          "codingCode": transaction["coding"]["code"],
          "acquiredDisposed": transaction["amounts"]["acquiredDisposedCode"],
          "shares": transaction["amounts"]["shares"],
          "sharePrice": sharePrice,
          "total": math.ceil(transaction["amounts"]["shares"] * sharePrice),
          "sharesOwnedFollowingTransaction": sharesOwnedFollowingTransaction
      }


      transactions.append({**base_data, **entry})

  return transactions



def flatten_filings(filings):
  unflattened_list = list(map(flatten_filing, filings))
  return [item for sublist in unflattened_list for item in sublist]

In [ ]:
transactions = flatten_filings(insider_trades_sample["transactions"])
trades = pd.DataFrame(transactions)
trades.head(5)

,periodOfReport,issuerCik,issuerTicker,reportingPersonName,reportingPersonCik,relationship,type,securityTitle,underlyingSecurity,codingCode,acquiredDisposed,shares,sharePrice,total,sharesOwnedFollowingTransaction
0,2024-07-25,1318605,TSLA,DENHOLM ROBYN M,1242782,"{'isDirector': True, 'isOfficer': False, 'isTe...",derivative,Non-Qualified Stock Option (right to buy),Common Stock,M,D,44096.0,0.00,0,0.0
1,2024-07-25,1318605,TSLA,DENHOLM ROBYN M,1242782,"{'isDirector': True, 'isOfficer': False, 'isTe...",derivative,Non-Qualified Stock Option (right to buy),Common Stock,M,D,58795.0,0.00,0,0.0
2,2024-07-25,1318605,TSLA,DENHOLM ROBYN M,1242782,"{'isDirector': True, 'isOfficer': False, 'isTe...",derivative,Non-Qualified Stock Option (right to buy),Common Stock,M,D,22861.0,0.00,0,0.0
3,2024-07-25,1318605,TSLA,DENHOLM ROBYN M,1242782,"{'isDirector': True, 'isOfficer': False, 'isTe...",derivative,Non-Qualified Stock Option (right to buy),Common Stock,M,D,10612.0,0.00,0,0.0
4,2024-07-25,1318605,TSLA,DENHOLM ROBYN M,1242782,"{'isDirector': True, 'isOfficer': False, 'isTe...",nonDerivative,Common Stock,NaN,M,A,44096.0,23.17,1021705,59096.0


In [ ]:
import json
from datetime import datetime, timedelta

def download_and_save_trades_per_year(year="2022"):
  log_file = open(f"trades_{year}.txt", "a")

  has_data = True
  start_from = 0
  total_filings_saved = 0
  last_count = 0
  date_format = "%Y-%m-%d"
  day = year + "-01-01"

  while has_data:
    insider_trades = insiderTradingApi.get_data({
      "query": {"query_string": {"query": f"periodOfReport:{day} AND issuer.tradingSymbol:*"}},
      "from": start_from,
      "size": "50",
      "sort": [{ "filedAt": { "order": "desc" } }]
    })

    if len(insider_trades["transactions"]) == 0:
      start_from = 0
      day_date = datetime.strptime(day, date_format) + timedelta(days=1)
      day = day_date.strftime(date_format)
      print(f'-- {day} --')

      if day == year + "-12-31":
        break

      continue

    total_filings_saved += len(insider_trades["transactions"])
    start_from += 50

    trades = flatten_filings(insider_trades["transactions"])

    for trade in trades:
      log_file.write(json.dumps(trade) + '\n')

    if total_filings_saved > last_count + 500:
      last_count = total_filings_saved
      print(f'{total_filings_saved } saved')

  log_file.close()
  print(f'{year} done. {total_filings_saved} total filings saved')


download_and_save_trades_per_year(year="2022")

550 saved
1100 saved
1621 saved
-- 2022-01-02 --
-- 2022-01-03 --
2151 saved
2701 saved
3251 saved
3801 saved
4351 saved
-- 2022-01-04 --


Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

# **For some reasons,my files couldn't be downloaded and it couldn't be processed but the following are the steps which I intended to run**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install jsonlines

In [3]:
import jsonlines
import datetime as dt

def load_trades_from_file_per_year(year="2022"):
  trades = []

  with jsonlines.open(f"/content/drive/MyDrive/Colab Notebooks/insider-trading-monitor/trades-{year}.txt") as reader:
    for trade in reader:
      trades.append(trade)

  trades_df = pd.DataFrame(trades)

  trades_df.drop('filingId', axis=1, inplace=True)

  trades_df["issuerTicker"] = trades_df["issuerTicker"].apply(lambda x : x.upper().replace(" ", ""))

  trades_df['periodOfReport'] = pd.to_datetime(trades_df['periodOfReport'])

  return trades_df

In [4]:
trades_2022 = load_trades_from_file_per_year(year="2022")
trades_2022.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/insider-trading-monitor/trades-2022.txt'

In [ ]:
def load_all_trades_from_drive():
  all_trades = pd.DataFrame()

  for year in range(2012, 2023):
    trades_per_year = load_trades_from_file_per_year(year=year)

    all_trades = pd.concat([all_trades, trades_per_year])

  return all_trades

all_trades = load_all_trades_from_drive()

# Data Cleaning

In [ ]:
filter_all = (all_trades["shares"] != all_trades["sharePrice"]) & \
  ( (all_trades["sharePrice"] < 6000) | (all_trades["shares"] == 1) ) & \
  (all_trades["total"] > 0) & \
  (all_trades["codingCode"] != "M") & \
  (all_trades["issuerTicker"] != "NONE") & \
  (all_trades["issuerTicker"] != "N/A") & \
  (all_trades["issuerTicker"] != "NA") & \
  (~all_trades["issuerCik"].str.contains("810893|1454510|1463208|1877939|1556801|827187")) # insider incorrectly reported share price

all_trades = all_trades[filter_all]

In [ ]:
[
    {
        "name": "Aaon Inc",
        "ticker": "AAON",
        "cik": "824142",
        "cusip": "000360206",
        "exchange": "NASDAQ",
        "category": "Domestic Common Stock",
        "sector": "Basic Materials",
        "industry": "Building Products & Equipment",
        "sic": "3585",
        "sicSector": "Manufacturing",
        "sicIndustry": "Air-Cond & Warm Air Heatg Equip & Comm & Indl Refrig Equip",
        "id": "7414f59fa8ea1454fe8f18f54ee80a9e"
    },
    ... more mappings
]

In [ ]:
def load_ticker_meta_data():
  nyse = pd.read_json('/content/drive/MyDrive/Colab Notebooks/insider-trading-monitor/nyse.json')
  nasdaq = pd.read_json('/content/drive/MyDrive/Colab Notebooks/insider-trading-monitor/nasdaq.json')

  nyse.drop(["cusip","sic","famaSector","famaIndustry","id", "currency", "location"], axis=1, inplace=True)
  nasdaq.drop(["cusip","sic","famaSector","famaIndustry","id", "currency", "location"], axis=1, inplace=True)

  nyse.rename(columns={'ticker': 'issuerTicker'}, inplace=True)
  nasdaq.rename(columns={'ticker': 'issuerTicker'}, inplace=True)

  return pd.concat([nyse, nasdaq])


ticker_meta_data = load_ticker_meta_data()
ticker_meta_data.head(5)

In [ ]:
all_trades = all_trades.merge(ticker_meta_data, \
                              on="issuerTicker", \
                              how="left", suffixes=(None,None))
all_trades = all_trades[all_trades["exchange"].notna()]

# Analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.style.use('seaborn')

params = {'legend.fontsize': '14',
          'font.size': '14',
          'axes.labelsize': '14',
          'axes.labelweight': 'bold',
          'axes.titlesize':'14',
          'xtick.labelsize':'14',
          'ytick.labelsize':'14'
         }

plt.rcParams.update(params)

# prettify axis: 2000000 to $2M
def millions_formatter(x, pos):
    return '$ {:,.0f} M'.format(x*1e-6)

In [ ]:
acquired_all = all_trades[all_trades["acquiredDisposed"]=="A"].groupby(['periodOfReport'])['total'].sum()
disposed_all = all_trades[all_trades["acquiredDisposed"]=="D"].groupby(['periodOfReport'])['total'].sum()

acquired_disposed_all = pd.merge(acquired_all, disposed_all, on='periodOfReport', how='outer')
acquired_disposed_all.rename(columns={'total_x': 'acquired', 'total_y': 'disposed'}, inplace=True)
acquired_disposed_all = acquired_disposed_all.sort_values(by=["periodOfReport"])

acquired_disposed_all.head(10)

In [ ]:
def plot_annual_graph(df):
  df.index = pd.to_datetime(df.index)

  acquired_yr = df.groupby(pd.Grouper(freq='Y'))['acquired'].sum()
  disposed_yr = df.groupby(pd.Grouper(freq='Y'))['disposed'].sum()

  acquired_disposed_yr = pd.merge(acquired_yr, disposed_yr, on='periodOfReport', how='outer')

  ax = acquired_disposed_yr.plot.bar(stacked=False, figsize=(15, 7), color=["#2196f3", "#ef5350"])

  ax.grid(True)
  ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(millions_formatter))
  ax.set_xticks(range(acquired_disposed_yr.index.size))
  ax.set_xticklabels([ts.strftime('%Y') for idx, ts in enumerate(acquired_disposed_yr.index)])
  ax.set_xlabel("Year")
  ax.set_ylabel("Amount $")
  ax.set_title("Acquired/Disposed per Year")
  ax.figure.autofmt_xdate(rotation=0, ha='center')

  plt.show()

plot_annual_graph(acquired_disposed_all)

# Distribution of Transaction Codes

In [ ]:
transaction_code = all_trades\
    .groupby(["acquiredDisposed", "codingCode"])['total'] \
    .sum()

ax_codes = transaction_code.plot.barh(figsize=(10,8))
ax_codes.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(millions_formatter))
ax_codes.set_xlabel("Amount $")
ax_codes.set_ylabel("Transaction Code")
ax_codes.set_title("Distribution of Transaction Codes (2012 - 2022)")
ax_codes.figure.autofmt_xdate(rotation=0, ha='center')

plt.show()

# Companies most often bought/sold in 2022

In [ ]:
year_22 = all_trades['periodOfReport'].dt.year == 2022

acquired_by_ticker = all_trades[(all_trades["acquiredDisposed"]=="A") & year_22] \
    .groupby(["issuerTicker"])['total'] \
    .sum() \
    .sort_values(ascending=False)

acquired_by_ticker.head(5)

In [ ]:
disposed_by_ticker = all_trades[(all_trades["acquiredDisposed"]=="D") & year_22] \
    .groupby(["issuerTicker"])['total'] \
    .sum() \
    .sort_values(ascending=False)

disposed_by_ticker.head(5)